# [1단계] 환경 설정

## 1-1. 필수 라이브러리 설치

In [ ]:
!pip install -q upstage ragas langchain langchain_openai langchain_community langchain-text-splitters pypdf datasets transformers torch accelerate

## 1-2. API 키 및 파일 준비

In [ ]:
import os
from google.colab import userdata
import glob

# Upstage API 키 설정
UPSTAGE_API_KEY = userdata.get('UPSTAGE_API_KEY')
print("Upstage API 키 설정이 완료되었습니다.")

# 허깅페이스 데이터셋 저장소 복제
hf_dataset_repo_url = "https://huggingface.co/datasets/sumilee/SKN14-Final-3Team-Data"
local_repo_path = "SKN14-Final-3Team-Data"
!git clone {hf_dataset_repo_url} {local_repo_path}

# 복제된 폴더 내의 모든 PDF 파일 경로 탐색
pdf_files = glob.glob(f"{local_repo_path}/**/*.pdf", recursive=True)
print(f"\n총 {len(pdf_files)}개의 PDF 파일을 발견했습니다.")

Upstage API 키 설정이 완료되었습니다.
Cloning into 'SKN14-Final-3Team-Data'...
remote: Enumerating objects: 233, done.
remote: Total 233 (delta 0), reused 0 (delta 0), pack-reused 233 (from 1)
Receiving objects: 100% (233/233), 543.61 KiB | 20.91 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Filtering content: 100% (202/202), 76.09 MiB | 17.31 MiB/s, done.

총 210개의 PDF 파일을 발견했습니다.


# [2단계] Upstage 파서로 고품질 텍스트 추출 (공식 문서 기반)

Upstage 공식 문서에 명시된 방법대로, requests 라이브러리를 사용하여 Document Parse API를 직접 호출합니다. 각 PDF를 처리하고, 반환된 JSON 결과에서 의미 있는 텍스트(paragraph, heading 등)만 추출하여 하나의 긴 텍스트로 합칩니다.

## 2-1. BeautifulSoup 라이브러리 설치

In [ ]:
# HTML 파싱을 위한 BeautifulSoup 라이브러리를 설치합니다.
!pip install -q beautifulsoup4

## 2-2. Upstage 문서 파싱

In [ ]:
import time
import json
import requests

UPSTAGE_ENDPOINT = "https://api.upstage.ai/v1/document-digitization"
UPSTAGE_MODEL_NAME = "document-parse"  # 필요시 "document-parse-v1.1" 등으로 교체만 하면 됨

def parse_document_with_upstage(pdf_path, max_retries=3, timeout=120):
    if not UPSTAGE_API_KEY:
        raise ValueError("UPSTAGE_API_KEY가 설정되지 않았어. 환경변수 또는 colab userdata에 넣어줘.")

    headers = {
        "Authorization": f"Bearer {UPSTAGE_API_KEY}",
        "Accept": "application/json",
    }

    # 간단한 지수 백오프 재시도
    backoff = 2.0
    for attempt in range(1, max_retries + 1):
        try:
            with open(pdf_path, "rb") as f:
                files = {"document": ("document.pdf", f, "application/pdf")}
                data = {"model": UPSTAGE_MODEL_NAME}
                resp = requests.post(
                    UPSTAGE_ENDPOINT,
                    headers=headers,
                    files=files,
                    data=data,
                    timeout=timeout,
                )
            # 상태코드 체크
            if resp.status_code >= 500 or resp.status_code == 429:
                # 서버/레이트리밋은 재시도
                if attempt < max_retries:
                    time.sleep(backoff)
                    backoff *= 2
                    continue
            resp.raise_for_status()

            try:
                return resp.json()  # 기대 스키마: {"elements": [...], ...}
            except json.JSONDecodeError:
                print(f"[경고] JSON 파싱 실패: {pdf_path}")
                return None

        except requests.RequestException as e:
            if attempt < max_retries:
                time.sleep(backoff)
                backoff *= 2
                continue
            print(f"[에러] 요청 실패({attempt}/{max_retries}): {pdf_path} - {e}")
            return None


## 2-3. 모든 PDF 파싱 및 HTML에서 텍스트 추출

In [ ]:
# --- Step 3: 파싱 결과에서 텍스트 추출 (개선 버전: 드롭인 교체) ---
from bs4 import BeautifulSoup
from tqdm import tqdm

# 선택 카테고리 화이트리스트 (기존 로직 유지)
_ALLOWED_CATEGORIES = {
    "paragraph", "heading1", "heading2", "heading3",
    "list_item", "table"
}

def _html_to_text_preserve_table(html: str) -> str:
    """문단/헤딩/목록은 평문으로, 표는 행/셀 구분을 보존해 텍스트로 변환."""
    if not html:
        return ""

    soup = BeautifulSoup(html, "html.parser")

    # 표 처리: <table> 있으면 행 단위로 탭/구분자 붙여 문자열화
    table = soup.find("table")
    if table:
        rows = []
        for tr in table.find_all("tr"):
            cells = []
            for cell in tr.find_all(["th", "td"]):
                # 셀 내부 텍스트
                cells.append(cell.get_text(separator=" ", strip=True))
            rows.append("\t".join(cells))  # 탭으로 셀 구분
        # 표 상단/하단에 구분선
        return "[TABLE]\n" + "\n".join(rows) + "\n[/TABLE]"

    # 표가 아니면 일반 텍스트 추출
    return soup.get_text(separator=" ", strip=True)

# 결과 누적: 기존 호환을 위해 all_parsed_text 유지 + 문서별 구조 추가
all_parsed_text = ""       # 기존 코드 호환
parsed_docs = []           # 새로 추가: 문서 단위 보존 (RAG에 권장)

for pdf_path in tqdm(pdf_files, desc="Upstage 파싱 및 텍스트 추출"):
    result_json = parse_document_with_upstage(pdf_path)

    if not result_json or "elements" not in result_json:
        print(f"[경고] 파싱 실패 또는 elements 없음: {pdf_path}")
        continue

    # 문서 단위로 chunk 모으기
    doc_chunks = []
    for element in result_json["elements"]:
        if element.get("category") in _ALLOWED_CATEGORIES:
            html_content = (element.get("content") or {}).get("html")
            text_content = _html_to_text_preserve_table(html_content)
            if text_content:
                doc_chunks.append(text_content)

    if not doc_chunks:
        print(f"[알림] 추출된 텍스트 없음: {pdf_path}")
        continue

    # 문서별 전체 텍스트
    doc_text = "\n".join(doc_chunks)

    # 기존 호환: all_parsed_text 이어붙이기
    all_parsed_text += doc_text + "\n"

    # RAG 친화적 구조 보존
    parsed_docs.append({
        "source": pdf_path,
        "chunks": doc_chunks,   # 패시지 단위
        "full_text": doc_text,  # 하나로 합친 본문
    })

print(f"\n총 {len(parsed_docs)}개 문서에서 텍스트 추출 완료.")


Ragas 평가를 위한 샘플 데이터가 준비되었습니다.


# [3단계] 훈련용/평가용 문서 청크 분리

Upstage 파서를 통해 생성된 전체 문서 청크(docs)를 훈련용과 평가용으로 분리합니다. 평가용으로는 전체 청크 중 일부(예: 50개)만 사용하여 평가 데이터셋을 생성하고, 나머지 대부분의 청크는 모델 훈련에 사용합니다. 이를 통해 데이터 유출(Data Leakage)을 원천적으로 방지합니다.

In [ ]:
import random

# 재현성을 위해 시드 고정
random.seed(42)

# 전체 docs를 섞은 뒤 일부를 평가용으로 선택
shuffled_docs = docs.copy()
random.shuffle(shuffled_docs)

docs_for_evaluation_generation = shuffled_docs[:50]   # 평가셋 생성용
docs_for_training = shuffled_docs[50:]               # 나머지는 훈련용

print(f"평가 데이터셋 생성용 청크: {len(docs_for_evaluation_generation)}개")
print(f"모델 훈련용 청크: {len(docs_for_training)}개")



--- 최종 평가 데이터셋 ---
Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 2
})


# [2단계 New] 기존의 데이터셋 활용해서 평가하기

In [ ]:
from datasets import load_dataset
from ragas import evaluate, EvaluationDataset
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall

# HF 허브에서 바로 불러오기
hf_ds = load_dataset("sssssungjae/finance-kb-mixed-dataset-final", split="validation")

# RAGAS용 데이터셋으로 변환
ragas_eval = EvaluationDataset.from_hf_dataset(hf_ds)

# 평가 실행
result = evaluate(
    dataset=ragas_eval,
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall],
)

df = result.to_pandas()
print(df.head())


# [4단계] Ragas로 평가 데이터셋 자동 생성

docs_for_evaluation_generation만을 사용하여, RAG 시스템을 평가하기 위한 (1) 질문, (2) 정답, (3) 근거 문서로 구성된 데이터셋을 자동으로 생성합니다.

## 4-1. TestsetGenerator 설정

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 데이터 생성에 사용할 LLM과 임베딩 모델 정의
generator_llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18")
critic_llm = ChatOpenAI(model="gpt-5-nano-2025-08-07") # 생성된 데이터 품질 검증용
embeddings = OpenAIEmbeddings()

# TestsetGenerator 생성
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
)

# 생성할 질문의 유형 정의
distributions = {
    simple: 0.5,
    reasoning: 0.25,
    multi_context: 0.25,
}

# (중요!) 훈련용 청크와 분리된, 평가 생성용 청크만 사용합니다.
evaluation_testset = generator.generate_with_langchain_docs(docs_for_evaluation_generation, test_size=20, distributions=distributions)

# 생성된 평가 데이터셋을 pandas 데이터프레임으로 변환
synthetic_eval_df = evaluation_testset.to_pandas()

print("\n--- 자동으로 생성된 최종 평가 데이터셋 (일부) ---")
display(synthetic_eval_df.head())

print("평가 데이터셋 생성을 위한 TestsetGenerator가 준비되었습니다.")

--- Langchain 기반 평가기 및 프롬프트 준비 완료 (수정 완료) ---


## 4-2. 평가 데이터셋 생성 실행

In [ ]:
import pandas as pd

# 20개의 평가 데이터셋을 생성합니다.
# docs 변수는 3단계에서 Upstage 파서로 만든 고품질 청크 리스트입니다.
testset = generator.generate_with_langchain_docs(docs, test_size=20, distributions=distributions)

# 생성된 데이터셋을 pandas 데이터프레임으로 변환하여 확인
synthetic_eval_df = testset.to_pandas()

print("\n--- 자동으로 생성된 평가 데이터셋 (일부) ---")
display(synthetic_eval_df.head())

100%|██████████| 2/2 [00:21<00:00, 10.61s/it]


--- 직접 구현한 RAG 평가 완료 ---

--- 최종 상세 평가 결과 ---


,question,answer,faithfulness,answer_relevancy
0,KB스타 개인형IRP(퇴직연금)의 운용/자산관리 수수료는 어떻게 되나요?,KB스타 개인형IRP의 운용 및 자산관리 수수료는 연 0.18%에서 0.20% 사이...,0,0
1,DC형 퇴직연금 가입자가 추가납입을 할 경우 세액공제 한도는 얼마인가요?,"DC형 퇴직연금 가입자가 추가납입을 할 경우, 연금저축과 합산하여 연간 최대 900...",0,0


# [5단계] 파인튜닝 모델로 답변 생성

이제 자동으로 생성된 '시험지'(synthetic_eval_df)를 우리가 파인튜닝한 모델에게 풀게 하여 '답안지'를 만듭니다.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 허브에 업로드한 원본 모델 ID
my_model_id = "sssssungjae/qwen3-8b-finance-full"

# 모델과 토크나이저 로딩
model = AutoModelForCausalLM.from_pretrained(my_model_id, device_map="auto", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(my_model_id)

print("--- 평가 대상 모델 로딩 완료 ---")

## 5-2. 모델 답변 생성 및 최종 평가 데이터셋 구축

In [ ]:
from tqdm import tqdm
from datasets import Dataset

generated_answers = []

# synthetic_eval_df에서 필요한 컬럼들을 리스트로 변환
questions = synthetic_eval_df['question'].tolist()

raw_contexts = synthetic_eval_df['contexts'].tolist()
raw_ground_truths = synthetic_eval_df['ground_truths'].tolist()

# ragas 요구 형식(List[List[str]])으로 보정
contexts = [[ctx] if isinstance(ctx, str) else ctx for ctx in raw_contexts]
ground_truths = [[gt] if isinstance(gt, str) else gt for gt in raw_ground_truths]

print("\n--- 우리 모델의 답변 생성 시작 ---")
for question, context in tqdm(zip(questions, contexts), total=len(questions)):
    # RAG 프롬프트 구성
    prompt = f"""Use the following CONTEXT to answer the QUESTION.
    CONTEXT:
    {"\n".join(context)}
    QUESTION:
    {question}
    """
    chat_prompt = tokenizer.apply_chat_template([{"role": "user", "content": prompt}], tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)

    # 답변 생성
    outputs = model.generate(**inputs, max_new_tokens=512) # 답변 길이를 넉넉하게
    answer = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    generated_answers.append(answer)

# 최종 평가 데이터셋 구성
final_eval_data = {
    'question': questions,
    'answer': generated_answers,
    'contexts': contexts,
    'ground_truth': ground_truths
}
final_eval_dataset = Dataset.from_dict(final_eval_data)

print("\n--- 최종 평가 데이터셋 구축 완료 ---")

# [6단계] Ragas로 최종 평가 실행 및 분석

모든 재료(자동 생성된 질문/문서/정답, 그리고 우리 모델이 생성한 답변)가 준비되었으니, Ragas의 evaluate 함수를 호출하여 우리 모델의 RAG 성능을 최종적으로 채점하고 분석합니다.

## 6-1. Ragas 평가 실행

evaluate 함수에 5단계에서 만든 final_eval_dataset과 측정할 지표들을 전달하여 평가를 실행합니다. 이 과정에서 Ragas는 내부적으로 OpenAI의 GPT 모델을 '심판'으로 사용하여 각 항목을 채점합니다.

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)

print("\n--- Ragas 최종 평가 시작 ---")
# evaluate 함수를 호출하여 평가를 시작합니다.
# final_eval_dataset 변수는 5단계에서 만들어졌습니다.
result = evaluate(
    dataset=final_eval_dataset,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
    ]
)

print("\n--- Ragas 평가 실행 완료 ---")

## 6-2. 최종 결과 확인 및 분석

평가 결과를 pandas 데이터프레임으로 변환하여, 각 지표에 대한 점수를 명확하게 확인하고 분석합니다. 이 표가 우리 모델의 최종 성적표가 됩니다.

In [ ]:
# 결과를 pandas 데이터프레임으로 변환하여 가독성을 높입니다.
result_df = result.to_pandas()

print("\n--- 최종 상세 평가 결과 ---")
display(result_df)

## 6-3. 최종 점수 요약

전체 데이터셋에 대한 각 지표의 평균 점수를 따로 출력하여, 우리 RAG 시스템의 전반적인 성능을 한눈에 파악할 수 있도록 요약합니다.


In [ ]:
print("\n--- 최종 평가 점수 (평균) ---")
# result 딕셔너리에서 각 지표의 평균 점수를 출력합니다.
# 소수점 셋째 자리까지 반올림하여 보기 좋게 만듭니다.
summary = {key: round(value, 3) for key, value in result.items()}
print(summary)